In [165]:
from astropy.time import Time
import datetime as dt
import astropy.units as u
import warnings
from astropy.utils.exceptions import AstropyUserWarning, AstropyWarning
from astropy.table import vstack, Table
import numpy as np
from transit_prediction.utils.list_of_constraints import List_of_constraints 

# Import astroplan tools and constraints
# Other constraints are available
from astroplan import (FixedTarget, Observer, EclipsingSystem,
                       PrimaryEclipseConstraint, is_event_observable,
                       AtNightConstraint, AltitudeConstraint,
                       TimeConstraint, AirmassConstraint, PhaseConstraint)

from masterfile import MasterFile

In [200]:
def min_start_time(constraints_list, obs, target, time, dt=1*u.min):
    
    time = Time(time)
    evening = obs.sun_set_time(time, which='previous')

    time_grid = np.arange(evening.jd, time.jd, dt.to('day').value) * u.d
    time_grid = Time(time_grid, format='jd')
    
    index = is_event_observable(constraints_list,
                                obs, target,
                                times=time_grid).squeeze()
    
    return time_grid[index][0]

def max_end_time(constraints_list, obs, target, time, dt=1*u.min):
    
    time = Time(time)
    morning = obs.sun_rise_time(time, which='next')

    time_grid = np.arange(time.jd, morning.jd, dt.to('day').value) * u.d
    time_grid = Time(time_grid, format='jd')
    
    index = is_event_observable(constraints_list,
                                obs, target,
                                times=time_grid).squeeze()
    
    return time_grid[index][-1]

def min_start_times(constraints_list, obs, target, times, dt=1*u.min):
    
    out = []
    for time in times:
        out.append(min_start_time(constraints_list, obs, target, time, dt=1*u.min))
        
    return Time(out)

def max_end_times(constraints_list, obs, target, times, dt=1*u.min):
    
    out = []
    for time in times:
        out.append(max_end_time(constraints_list, obs, target, time, dt=1*u.min))
        
    return Time(out)

# Get planets info

In [2]:
target_list = ['51 Peg', 'ups And b']  # Stefan

In [3]:
data = MasterFile.read().by_plName(*target_list)

# Inputs

In [222]:
t1, t2 = Time(['2019-08-01 00:00', '2020-01-31 00:00'])
site = 'cfht'
# abs_baseline = 30 * u.min
# baseline = None #2
# See astroplan available constraints
constraints = [AtNightConstraint.twilight_nautical(),
               AirmassConstraint(max=2.),
               TimeConstraint(t1,t2)
              ]
constraints_list = List_of_constraints(constraints)
supp_cols = ['st_j','st_h']
n_eclipses = 5000

## Special inputs for phase

In [234]:
# phase_range = [0.40, 0.48]  # One range at the time
phase_range = [0.52, 0.60]
obs_time = 5 * u.h
dt_grid = 0.5*u.h

In [ ]:
# constraints_list = [*constraints, PhaseConstraint()]

# Add info about planets

In [11]:
from transit_prediction.utils.table_edited import Column 

In [12]:
phase_zero = [2458708.83240,2458710.05353] * u.d

In [13]:
data.add_column(Column(phase_zero, name='pl_phase_curve_zero', 
       description='equivalent to mid-transit time for a non eclipsing system. Use with caution')
               )

In [14]:
required_info = data['pl_name','pl_orbper','pl_phase_curve_zero']

In [15]:
required_info.show_in_notebook()

idx,pl_name,pl_orbper,pl_phase_curve_zero
,,d,d
0,51 Peg b,4.230785,2458708.8324
1,ups And b,4.617033,2458710.05353


In [225]:
# Define needed quantities based on planets infos
# Must be quatities arrays (astropy)
# Here we use a given astropy Table (data) to get the infos

epoch, period = data.cols_2_qarr('pl_phase_curve_zero', 'pl_orbper')
epoch = Time(epoch, format='jd')
pl_name = data['pl_name']


observing_time = t1
obs = Observer.at_site(site)

In [235]:
col_names = ('pl_name',
             'Obs_start',
             'Phase_start',
             'Obs_end',
             'Phase_end',
             'mid_tr',
             'AM_mid_tr',
             'moon',
             *supp_cols
            )
meta = {'Time_limits': [t1, t2],
        'Target_list': pl_name.tolist(),
        'Site': site,
        'Phase_range': phase_range,
        **constraints_list.show()
       }

In [236]:
from astroplan import PeriodicEvent

In [237]:
full_table = Table()
for itar in range(len(pl_name)):

    # -------------------------
    # Steps to predict transits
    # -------------------------
    try:
        target = FixedTarget.from_name(pl_name[itar])
    except NameResolveError as e:
        print(e)
        try_name = ' '.join(pl_name[itar].split(' ')[:-2])
        print("Trying with {}".format(try_name))
        try:
            target = FixedTarget.from_name(try_name)
        except NameResolveError as e:
            print(e)
            print('Searching in masterfile for RA and dec')
            try_radec = MasterFile.read().by_plName(pl_name[itar])
            ra = try_radec['ra'].quantity
            dec = try_radec['dec'].quantity
            coord = SkyCoord(ra=ra, dec=dec)
            target = FixedTarget(coord=coord,
                                 name=pl_name[itar])
            

    
    dt_grid = 0.5*u.h
    d_phase = (dt_grid /period[itar]).decompose().value
    [p1, p2] = phase_range
    p1 += d_phase/10  # Make sure it's not on the boundary
    p2 -= d_phase/10
    phase_grid = np.arange(p1, p2, d_phase)
    phase_grid = phase_grid*period[itar] + epoch[itar]
    
    t_grid = []
    for phase in phase_grid:
        sys = EclipsingSystem(primary_eclipse_time=phase,
                              orbital_period=period[itar])
        t_temp = sys.next_primary_eclipse_time(observing_time,
                                               n_eclipses=n_eclipses)
        t_grid.append(t_temp.jd)
    t_grid = Time(t_grid, format='jd').T

    
    if t_grid[-1,-1] < t2:
        warnings.warn('end time ('+t2.value +
                      ') is passed the last computed event time (' +
                      t_mid[-1].value+')\n' +
                     '\t You can change the n_eclipse kwarg ' +
                     'value or choose a different window (start or end time)',
                     AstropyUserWarning
                    )

    t_grid = t_grid[(t_grid < t2).any(axis=1)]
    
    events = []
    for grid in t_grid:
        index = is_event_observable(constraints_list, obs, target, times=grid).squeeze()
        if index.any():
            events.append(np.mean(grid[index].jd))
    events = Time(events, format='jd')
    
    # Finally add phase constraint
    sys = PeriodicEvent(epoch=epoch[itar], period=period[itar])
    
    final_constraints = [*constraints_list,
        PhaseConstraint(sys, *phase_range)]
    
    obs_start = min_start_times(final_constraints, obs, target, events)
    obs_end = max_end_times(final_constraints, obs, target, events)
    baseline = obs_end - obs_start
    t_mid = obs_start + baseline/2
    
    index = (obs_end - obs_start) > obs_time

    # -------------------
    # End of steps to predict transits
    # -------------------

    # Put the infos in a table and stack it to the full table
    if index.any():
        name = np.repeat(target.name,index.sum()).astype(str)
        moon = obs.moon_illumination(t_mid[index])
        phase_start = sys.phase(obs_start[index])
        phase_end = sys.phase(obs_end[index])
        AM_mid = obs.altaz(t_mid[index], target).secz
        supp = [np.repeat(data[key][itar],index.sum())
                for key in supp_cols]
        cols = [name,
                obs_start[index].iso,
                phase_start,
                obs_end[index].iso,
                phase_end,
                t_mid[index].iso,
                AM_mid,
                moon,
                *supp
               ]
        table_sys = Table(cols, names=col_names, masked=True)
        full_table = vstack([table_sys, full_table])
    else:
        warnings.warn('No event found for '+sys.name, AstropyUserWarning)

if full_table:
    full_table.sort('mid_tr')
    full_table.meta = meta
else:
    warnings.warn('No event found at all', AstropyUserWarning)

In [238]:
full_table

pl_name,Obs_start,Phase_start,Obs_end,Phase_end,mid_tr,AM_mid_tr,moon,st_j,st_h
,,,,,,,,,
str9,str23,float64,str23,float64,str23,float64,float64,float64,float64
51 Peg b,2019-08-11 07:44:54.867,0.5250189409411916,2019-08-11 15:10:20.244,0.5981310196512811,2019-08-11 11:27:37.555,1.0099964112005178,0.8535582052146329,4.655,4.234
ups And b,2019-08-12 10:10:17.418,0.5387259969493939,2019-08-12 15:10:47.399,0.5839239276898928,2019-08-12 12:40:32.408,1.190385819343101,0.9189929129281438,3.175,2.957
51 Peg b,2019-08-28 06:38:35.389,0.5322992041391231,2019-08-28 13:30:39.540,0.5999365687200324,2019-08-28 10:04:37.465,1.021750347543217,0.0614021445437164,4.655,4.234
ups And b,2019-09-04 10:08:41.249,0.5200390912252708,2019-09-04 15:18:25.655,0.5666268643269863,2019-09-04 12:43:33.452,1.0801805508819908,0.33158424484289867,3.175,2.957
51 Peg b,2019-09-14 05:31:45.069,0.5394950928403798,2019-09-14 11:39:58.836,0.5999365689605552,2019-09-14 08:35:51.953,1.0455544179906504,0.9981009841299522,4.655,4.234
ups And b,2019-09-18 07:44:05.611,0.5305411853787942,2019-09-18 15:22:00.608,0.5994160789082402,2019-09-18 11:33:03.109,1.0876574205537648,0.8377881883090996,3.175,2.957
ups And b,2019-10-02 06:49:25.855,0.5545697897876823,2019-10-02 11:50:35.562,0.5998673073879096,2019-10-02 09:20:00.708,1.1904708887024427,0.17800626236652828,3.175,2.957
51 Peg b,2019-10-05 07:14:48.948,0.5200300679477109,2019-10-05 12:41:37.956,0.5736740406630009,2019-10-05 09:58:13.452,1.0771593086786837,0.47267857834165666,4.655,4.234


In [239]:
file_name = '/home/adb/Doctorat/transit_predictions/list_of_phase_stefan_{}_{}.ecsv'

In [240]:
full_table.write(file_name.format(*meta['Phase_range']),
                 delimiter=',')